In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
file_path = r'C:\Users\user\Documents\Data Analytics Path\DSTI\Pthyton and Machine Leaning Lab\Project 1\Project 1\books.v3.2.dropped_columns.xlsx'
df = pd.read_excel(file_path)

In [4]:
df.head(10)

,bookID,authors,average_rating,isbn13,language_code,num_pages,ratings_count,text_reviews_count,publication_date,publisher
0,12224,Sam Bass Warner,3.58,9780674842113,en-US,236,61,6,38097,Harvard University Press
1,16914,David E. Smith (Turgon of TheOneRing.net,3.58,9781593600112,eng,400,26,4,38083,Cold Spring Press
2,26012,Bret Easton Ellis,3.46,9782264031914,fre,537,56,5,36937,2018-10-01 00:00:00
3,28225,Nicholson Baker,3.84,9782264017697,fre,191,6,1,36117,2018-10-01 00:00:00
4,22128,James Wesley,3.63,9781563841552,eng,342,38,4,36175,Huntington House Publishers
5,31095,Robert M. Pirsig,3.58,9781403356208,en-GB,600,54,5,37635,1st Book Library
6,5478,George Orwell,4.18,9781421808321,eng,387,215,17,38534,1st World Library
7,31373,Elizabeth George,4.10,9780553575101,eng,718,10608,295,36831,Bantam Books
8,16285,Terry Deary,4.40,9780713659894,eng,62,5,0,37195,A & C Black
9,18843,Virginia Woolf,4.07,9780156028066,eng,216,244,13,37711,A Harvest Book


In [5]:
df = df.drop(columns=["isbn13"])

In [9]:
df.describe()

,bookID,average_rating,num_pages,ratings_count,text_reviews_count,publication_date
count,11127.000000,11127.000000,11127.000000,1.112700e+04,11127.000000,11127.000000
mean,21310.938887,3.933631,336.376921,1.793641e+04,541.854498,36768.842995
std,13093.358023,0.352445,241.127305,1.124794e+05,2576.176608,2998.495083
min,1.000000,0.000000,0.000000,0.000000e+00,0.000000,4750.000000
25%,10287.000000,3.770000,192.000000,1.040000e+02,9.000000,35992.500000
50%,20287.000000,3.960000,299.000000,7.450000e+02,46.000000,37681.000000
75%,32104.500000,4.135000,416.000000,4.993500e+03,237.500000,38625.000000
max,45641.000000,5.000000,6576.000000,4.597666e+06,94265.000000,43921.000000


In [11]:
df.isna().sum()

bookID                0
authors               0
average_rating        0
language_code         0
  num_pages           0
ratings_count         0
text_reviews_count    0
publication_date      0
publisher             0
dtype: int64

In [13]:
print(df.columns)

Index(['bookID', 'authors', 'average_rating', 'language_code', '  num_pages',
       'ratings_count', 'text_reviews_count', 'publication_date', 'publisher'],
      dtype='object')


In [25]:
import category_encoders as ce
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import MinMaxScaler

# 1. Split the data into training and test sets
X = df[['bookID', 'authors', 'publisher', '  num_pages', 'ratings_count', 'text_reviews_count','language_code']]  # Features
y = df['average_rating']  # Target

# Use train_test_split to ensure no leakage
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [22]:
pip install category_encoders

Note: you may need to restart the kernel to use updated packages.


In [27]:
# 2. Fit the TargetEncoder on 'authors', 'publisher' and 'language_code' from the training set
target_encoder = ce.TargetEncoder(cols=['authors', 'publisher','language_code'])

# Fit on the training set
X_train[['authors_encoded', 'publisher_encoded', 'language_encoded']] = target_encoder.fit_transform(X_train[['authors', 'publisher','language_code']], y_train)

# Apply the encoder to the test set
X_test[['authors_encoded', 'publisher_encoded', 'language_encoded']] = target_encoder.transform(X_test[['authors', 'publisher','language_code']])

In [30]:
from sklearn.preprocessing import FunctionTransformer

# 3. Apply log transformation on review_count to reduce skewness and then calculate relative log-transformed counts

# Step 1: Log-transform ratings_count using np.log1p (log(x + 1) for non-negative data)
log_transformer = FunctionTransformer(np.log1p, validate=True)

# Step 2: Fit-transform log-transformed counts on training data
X_train['log_ratings_count'] = log_transformer.fit_transform(X_train[['ratings_count']])

# Step 3: Apply same transformation to the test data
X_test['log_ratings_count'] = log_transformer.transform(X_test[['ratings_count']])

# Step 4: Normalize log-transformed counts by dividing by the total sum in the training set
total_log_ratings_count = X_train['log_ratings_count'].sum()

X_train['relative_log_ratings_count'] = X_train['log_ratings_count'] / total_log_ratings_count
X_test['relative_log_ratings_count'] = X_test['log_ratings_count'] / total_log_ratings_count  # Note: keep using total from training

print(X_train.head())
print(X_test.head())

       bookID         authors                                  publisher  \
4001     1097  Eric Schlosser                           Harper Perennial   
10605     639  Kevin O'Malley                           Walker & Company   
6606     1362       Herodotus                              Penguin Books   
7889    11021  Jane E. Gerver       Random House Books for Young Readers   
5391    41861      Lane Smith  Little  Brown & Company for Young Readers   

         num_pages  ratings_count  text_reviews_count language_code  \
4001           399         190039                4922           eng   
10605           32           1773                 274           eng   
6606           716          34727                 597           eng   
7889           112            140                   8           eng   
5391            40           3500                 393           eng   

       authors_encoded  publisher_encoded  language_encoded  \
4001          3.905776           3.897782          3.

In [34]:
# 4. Train the GradientBoostingRegressor using the weights
X_train_final = X_train[['  num_pages', 'authors_encoded', 'publisher_encoded','language_encoded']]  # Final training features
X_test_final = X_test[['  num_pages', 'authors_encoded', 'publisher_encoded','language_encoded']]  # Final test features
print(X_train_final.head())
print(X_test_final.head())

         num_pages  authors_encoded  publisher_encoded  language_encoded
4001           399         3.905776           3.897782          3.933261
10605           32         3.952669           3.891874          3.933261
6606           716         3.942332           3.906212          3.933261
7889           112         3.950067           3.945729          3.933261
5391            40         3.969583           3.919635          3.933261
        num_pages  authors_encoded  publisher_encoded  language_encoded
8663          144         3.904529           3.932207          3.933261
483           400         3.894729           3.866572          3.933261
8403         1016         3.908473           3.819788          3.933261
6382          240         3.975987           3.902523          3.933261
1844          192         3.933624           3.979367          3.933261


In [36]:
model = GradientBoostingRegressor()
model.fit(X_train_final, y_train, sample_weight=X_train['relative_log_ratings_count'])

# Make predictions on the test set
y_pred = model.predict(X_test_final)

#Evaluation with Metrics RMSE and MAE

from sklearn.metrics import mean_squared_error

mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5
print(f'RMSE: {rmse}')

RMSE: 0.32683492262086344


In [43]:
# Import necessary libraries
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Initialize the Random Forest Regressor
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)

# Fit the model to the training data
rf_model.fit(X_train_final, y_train, sample_weight=X_train['relative_log_ratings_count'])

# Make predictions on the test data
y_pred_rf = rf_model.predict(X_test_final)

# Evaluate the model
mse_rf = mean_squared_error(y_test, y_pred_rf)
r2_rf = r2_score(y_test, y_pred_rf)

print(f"Random Forest - Mean Squared Error (MSE): {mse_rf}")
print(f"Random Forest - R-squared (R2 Score): {r2_rf}")

Random Forest - Mean Squared Error (MSE): 0.10756325664906011
Random Forest - R-squared (R2 Score): 0.16198096787453764


In [ ]:
# 3. Normalize review counts for weighting
scaler = MinMaxScaler()

# Combine ratings_count and text_reviews_count for better weight determination
X_train['combined_reviews'] = X_train['ratings_count'] + X_train['text_reviews_count']
X_test['combined_reviews'] = X_test['ratings_count'] + X_test['text_reviews_count']

# Normalize combined_reviews
X_train['combined_reviews_normalized'] = scaler.fit_transform(X_train[['combined_reviews']])
X_test['combined_reviews_normalized'] = scaler.transform(X_test[['combined_reviews']])

